# Reverse engineering Mac OS (And iPhone) screen time SQLite db

Based on:
1. [Knowledge is Power! Using the macOS/iOS knowledgeC.db Database to Determine Precise User and Application Usage](https://www.mac4n6.com/blog/2018/8/5/knowledge-is-power-using-the-knowledgecdb-database-on-macos-and-ios-to-determine-precise-user-and-application-usage)
2. [Knowledge is Power II – A Day in the Life of My iPhone using knowledgeC.db](https://www.mac4n6.com/blog/2018/9/12/knowledge-is-power-ii-a-day-in-the-life-of-my-iphone-using-knowledgecdb)


In [1]:
import sqlite3
import os
import pandas as pd

In [2]:
DB_PATH = os.path.join(os.path.expanduser("~"), "Application Support", "Knowledge", "knowledgeC.db")
DB_PATH_DUMP = os.path.join(os.path.expanduser("~"), "knowledgeC_dump20240328.db")

In [3]:
def query_and_fetchall(db_path:str, query:str) -> pd.DataFrame:
    conn = sqlite3.connect(db_path)
    return pd.read_sql_query(query, conn)


### Listing all the tables

In [14]:
q_all_table_names = "SELECT name FROM sqlite_master WHERE type='table'"
df_tables = query_and_fetchall(DB_PATH_DUMP, q_all_table_names)

df_tables

,name
0,ZADDITIONCHANGESET
1,ZCONTEXTUALCHANGEREGISTRATION
2,ZCONTEXTUALKEYPATH
3,ZCUSTOMMETADATA
4,Z_4EVENT
5,ZDELETIONCHANGESET
6,ZHISTOGRAM
7,ZHISTOGRAMVALUE
8,ZKEYVALUE
9,ZOBJECT


### See all /app/usage events (note that data is a bit different from the post)

In [15]:
q_app_usage = """
SELECT
  datetime(ZOBJECT.ZCREATIONDATE+978307200,'UNIXEPOCH', 'LOCALTIME') as "event_datetime", 
  CASE ZOBJECT.ZSTARTDAYOFWEEK 
      WHEN "1" THEN "Sunday"
      WHEN "2" THEN "Monday"
      WHEN "3" THEN "Tuesday"
      WHEN "4" THEN "Wednesday"
      WHEN "5" THEN "Thursday"
      WHEN "6" THEN "Friday"
      WHEN "7" THEN "Saturday"
  END "event_dow",
  ZOBJECT.ZSECONDSFROMGMT/3600 AS "event_gtm_offset",
  datetime(ZOBJECT.ZSTARTDATE+978307200,'UNIXEPOCH', 'LOCALTIME') as "event_start", 
  datetime(ZOBJECT.ZENDDATE+978307200,'UNIXEPOCH', 'LOCALTIME') as "event_end", 
  (ZOBJECT.ZENDDATE-ZOBJECT.ZSTARTDATE) as "event_duration",
  ZOBJECT.ZSTREAMNAME event_type, 
  ZOBJECT.ZVALUESTRING event_description,
  ZOBJECT.ZHASCUSTOMMETADATA has_custom_metadadata, 
  ZOBJECT.ZHASSTRUCTUREDMETADATA has_structured_metadata, 
  ZOBJECT.ZSTRING event_zstring, 
  ZOBJECT.ZVALUECLASS event_valueclass, 
  ZSOURCE.ZDEVICEID event_source_dev_id 
FROM ZOBJECT
  LEFT JOIN ZSOURCE ON ZOBJECT.ZSOURCE  = ZSOURCE.Z_PK
  LEFT JOIN ZSTRUCTUREDMETADATA on ZOBJECT.ZSTRUCTUREDMETADATA = ZSTRUCTUREDMETADATA.Z_PK
WHERE ZSTREAMNAME = "/app/usage" 
ORDER BY "START"
"""

In [26]:
df_app_usage = query_and_fetchall(DB_PATH_DUMP, q_app_usage)
df_app_usage[2300:2500]

,ENTRY CREATION,DAY OF WEEK,GMT OFFSET,START,END,USAGE IN SECONDS,ZSTREAMNAME,ZVALUESTRING,ZHASCUSTOMMETADATA,ZHASSTRUCTUREDMETADATA,ZSTRING,ZVALUECLASS,ZDEVICEID
2300,2024-03-08 14:44:28,Friday,-3,2024-03-08 14:44:24,2024-03-08 14:44:27,3,/app/usage,com.microsoft.VSCode,0,1,None,1,None
2301,2024-03-08 14:44:29,Friday,-3,2024-03-08 14:44:27,2024-03-08 14:44:29,2,/app/usage,com.google.Chrome,0,1,None,1,None
2302,2024-03-08 14:47:32,Friday,-3,2024-03-08 14:44:29,2024-03-08 14:47:32,183,/app/usage,com.microsoft.VSCode,0,1,None,1,None
2303,2024-03-08 14:48:05,Friday,-3,2024-03-08 14:47:32,2024-03-08 14:48:05,33,/app/usage,com.google.Chrome,0,1,None,1,None
2304,2024-03-08 14:48:06,Friday,-3,2024-03-08 14:48:05,2024-03-08 14:48:06,1,/app/usage,com.microsoft.VSCode,0,1,None,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2024-03-08 19:50:20,Friday,-3,2024-03-08 19:47:39,2024-03-08 19:50:20,161,/app/usage,com.burbn.instagram,0,1,None,1,380D4F66-0F14-52B1-85FB-B8B9188119BA
2496,2024-03-08 20:09:06,Friday,-3,2024-03-08 19:58:32,2024-03-08 20:09:06,634,/app/usage,com.google.Chrome,0,1,None,1,None
2497,2024-03-08 20:00:26,Friday,-3,2024-03-08 19:59:57,2024-03-08 20:00:26,29,/app/usage,net.whatsapp.WhatsApp,0,1,None,1,380D4F66-0F14-52B1-85FB-B8B9188119BA
2498,2024-03-08 20:02:31,Friday,-3,2024-03-08 20:00:26,2024-03-08 20:02:31,125,/app/usage,com.burbn.instagram,0,1,None,1,380D4F66-0F14-52B1-85FB-B8B9188119BA


In [5]:
q_all_events = """
SELECT
  ZOBJECT.Z_PK AS event_id,
  datetime(ZOBJECT.ZCREATIONDATE+978307200,'UNIXEPOCH', 'LOCALTIME') AS event_datetime, 
  CASE ZOBJECT.ZSTARTDAYOFWEEK 
      WHEN "1" THEN "Sunday"
      WHEN "2" THEN "Monday"
      WHEN "3" THEN "Tuesday"
      WHEN "4" THEN "Wednesday"
      WHEN "5" THEN "Thursday"
      WHEN "6" THEN "Friday"
      WHEN "7" THEN "Saturday"
  END AS event_dow,
  ZOBJECT.ZSECONDSFROMGMT/3600 AS event_gtm_offset,
  datetime(ZOBJECT.ZSTARTDATE+978307200,'UNIXEPOCH', 'LOCALTIME') AS event_start, 
  datetime(ZOBJECT.ZENDDATE+978307200,'UNIXEPOCH', 'LOCALTIME') AS event_end, 
  (ZOBJECT.ZENDDATE-ZOBJECT.ZSTARTDATE) AS event_duration,
  ZOBJECT.ZSTREAMNAME AS event_type, 
  ZOBJECT.ZVALUESTRING AS event_description,

  ZOBJECT.ZHASCUSTOMMETADATA AS has_custom_md, 
  ZCUSTOMMETADATA.ZNAME AS custom_md_name,
  ZCUSTOMMETADATA.ZDOUBLEVALUE AS custom_md_double_value,
  ZCUSTOMMETADATA.ZSTRINGVALUE AS custom_md_string_value,
  ZOBJECT.ZHASSTRUCTUREDMETADATA AS has_structured_md, 
  ZSTRUCTUREDMETADATA.Z_DKINTENTMETADATAKEY__INTENTVERB AS structured_md_intent_verb, 
  ZSTRUCTUREDMETADATA.Z_DKINTENTMETADATAKEY__INTENTCLASS AS structured_md_intent_class, 
  ZSTRUCTUREDMETADATA.Z_DKINTENTMETADATAKEY__DERIVEDINTENTIDENTIFIER AS structured_md_derived_intent_id,
  ZSTRUCTUREDMETADATA.Z_CDENTITYMETADATAKEY__NAME AS structured_md_entity_name,
  ZSTRUCTUREDMETADATA.Z_DKNOTIFICATIONUSAGEMETADATAKEY__BUNDLEID AS structured_md_bundle_id,
  HEX(ZSTRUCTUREDMETADATA.Z_DKINTENTMETADATAKEY__SERIALIZEDINTERACTION) AS structured_md_serialized_interaction,
  --ZOBJECT.ZSTRING AS event_zstring, 
  ZOBJECT.ZVALUECLASS AS event_valueclass, 
  ZSOURCE.ZDEVICEID AS source_dev_id ,
  ZSOURCE.ZGROUPID as source_group_id,
  ZSOURCE.ZITEMID AS source_item_id,
  ZSOURCE.ZBUNDLEID AS  source_boundle_id,
  CURRENT_TIMESTAMP AS extraction_dt
FROM source.ZOBJECT
  LEFT JOIN source.ZSOURCE ON source.ZOBJECT.ZSOURCE  = source.ZSOURCE.Z_PK
  LEFT JOIN source.ZSTRUCTUREDMETADATA ON source.ZOBJECT.ZSTRUCTUREDMETADATA = source.ZSTRUCTUREDMETADATA.Z_PK
  LEFT JOIN source.Z_4EVENT ON source.ZOBJECT.Z_PK = source.Z_4EVENT.Z_11EVENT
  LEFT JOIN source.ZCUSTOMMETADATA ON source.Z_4EVENT.Z_4CUSTOMMETADATA = source.ZCUSTOMMETADATA.Z_PK
"""

In [ ]:
df_app_usage = query_and_fetchall(DB_PATH_DUMP, q_all_events)

In [6]:
import duckdb

con = duckdb.connect("ddb_test_20240331.duckdb")
con.sql("INSTALL sqlite;")
con.sql("LOAD sqlite;")



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [20]:
con.sql(f"ATTACH '{DB_PATH_DUMP}' AS sourcedb2 (TYPE SQLITE);")

In [23]:
con.sql("USE sourcedb2;")

In [27]:
results = duckdb.sql("SHOW ALL TABLES").df()

In [28]:
results

,database,schema,name,column_names,column_types,temporary


In [7]:
results = duckdb.sql(q_all_events).df()

CatalogException: Catalog Error: Table with name ZOBJECT does not exist!
Did you mean "temp.information_schema.tables"?

In [ ]:
import sqlite3

def copytransfer_data(destination_db_path:str, destination_table:str, source_db_path:str, source_query:str, pk_column:str):
    source_conn = sqlite3.connect(source_db_path)
    source_cursor = source_conn.cursor()
    
    source_cursor.execute(source_query)
    data = source_cursor.fetchall()
    column_names = [description[0] for description in source_cursor.description]
    
    dest_conn = sqlite3.connect(destination_db_path)
    dest_cursor = dest_conn.cursor()
    
    dest_cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{destination_table}';")
    if dest_cursor.fetchone() is None:
        columns = ', '.join([f"{col} TEXT" for col in column_names])
        dest_cursor.execute(f"CREATE TABLE destination_table (event_id INTEGER PRIMARY KEY, {columns})")
    else:
        # Optional: Clear the table if you strictly want to overwrite every time
        dest_cursor.execute("DELETE FROM destination_table")
    
    # Prepare the insert or replace statement
    placeholders = ', '.join(['?' for _ in column_names])
    dest_cursor.execute(f"PRAGMA table_info(destination_table)")
    dest_columns = dest_cursor.fetchall()
    
    # Adjust for any missing columns in destination_table
    for col in column_names:
        if col not in [d[1] for d in dest_columns]:
            dest_cursor.execute(f"ALTER TABLE destination_table ADD COLUMN {col} TEXT")
    
    insert_or_replace_query = f"INSERT OR REPLACE INTO destination_table ({', '.join(column_names)}) VALUES ({placeholders})"
    
    # Insert or replace data into the destination table
    dest_cursor.executemany(insert_or_replace_query, data)
    
    # Commit changes and close connections
    dest_conn.commit()
    source_conn.close()
    dest_conn.close()

# Example usage
source_db_path = 'source_db.db' # Path to your source database
destination_db_path = 'destination_db.db' # Path to your destination database
source_query = 'SELECT * FROM source_table' # Your query here

transfer_data(source_db_path, destination_db_path, source_query)